In [16]:
# ! pip install datasets
# ! pip install transformers
# ! pip install accelerate
# ! pip install trl
# ! pip install peft

In [6]:
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer, DataCollatorForLanguageModeling
from datasets import Dataset, DatasetDict
import torch
import time

In [2]:
# Load dataset
from experiments.model_training.prepare_dataset import OpenAICompletionDataPoint
dataset_path = "data/test_full-v1.jsonl"
dataset = []
with open(dataset_path, "r") as f:
    for line in f:
        datapoint = OpenAICompletionDataPoint.model_validate_json(line)
        dataset.append(datapoint)
print(len(dataset))

2025-08-04 13:51:53.567 | INFO     | tau2.utils.utils:<module>:27 - Using data directory from source: /lambda/nfs/victor-north-tx/tau2-bench-private/data


2025-08-04 13:51:54.626 | INFO     | tau2.utils.llm_utils:<module>:43 - VLLM_API_BASE: http://127.0.0.1:8000/v1
2025-08-04 13:51:54.627 | INFO     | tau2.utils.llm_utils:<module>:44 - OLLAMA_API_BASE: http://127.0.0.1:11434
2025-08-04 13:51:54.627 | INFO     | tau2.utils.llm_utils:<module>:78 - LiteLLM: Cache is disabled
2025-08-04 13:51:54.627 | WARNING  | tau2.utils.llm_utils:<module>:85 - Sonnet thinking is disabled
2025-08-04 13:51:54.710 | DEBUG    | tau2.registry:<module>:194 - Registering default components...
2025-08-04 13:51:54.710 | DEBUG    | tau2.registry:<module>:236 - Default components registered successfully. Registry info: {
  "domains": [
    "mock",
    "airline",
    "retail",
    "telecom",
    "telecom-workflow"
  ],
  "agents": [
    "llm_agent",
    "llm_agent_gt",
    "llm_agent_solo",
    "llm_agent_completion",
    "llm_agent_gt_completion",
    "llm_agent_solo_completion"
  ],
  "users": [
    "user_simulator",
    "dummy_user"
  ],
  "task_sets": [
    "moc

1107


In [3]:
dp0: OpenAICompletionDataPoint = dataset[0]
print(type(dp0))
print(dp0.parallel_tool_calls)
print(len(dp0.messages))
print(len(dp0.tools))
for i, message in enumerate(dp0.messages):
    if message["role"] in ["system", "tool"]:
        continue
    print(f"{i} {message['role']}: {message.get('content', 'tool call...')}")

<class 'experiments.model_training.prepare_dataset.OpenAICompletionDataPoint'>
True
18
14
1 assistant: Hi! How can I help you today?
2 user: Hi, I need to change my upcoming flight from JFK. My cat is really sick and I need to get home sooner. Can you help me look into changing my flight to an earlier nonstop option?
3 assistant: To assist you with changing your flight, I need to know your user ID and the reservation ID for the flight you want to change. Could you please provide those? If you don't know your reservation ID, I can help you locate it.
4 user: I don’t have my reservation ID handy, but my user ID is daiki_lee_6144. Can you look up my reservation with that?
5 assistant: tool call...
7 assistant: You have three reservations under your user ID daiki_lee_6144: DF89BM, COVE6R, and IIHXDG.

Could you please specify which reservation you want to change? If you are not sure, you can provide more details about the flight such as the destination or date, and I can help identify the 

In [ ]:
# Load model and tokenizer

# Model checkpoint
# MODEL_NAME = "Qwen/Qwen2.5-3B-Instruct"
# MODEL_NAME = "Qwen/Qwen2.5-0.5B"


## NOTE: Use instruct models! Non instruct models require better generation stopping strategies.
# MODEL_NAME = "/home/ubuntu/victor-north-tx/models/Qwen2.5-0.5B"
# MODEL_NAME = "/home/ubuntu/victor-north-tx/models/Qwen2.5-3B"
# MODEL_NAME = "/home/ubuntu/victor-north-tx/models/Qwen2.5-7B"
MODEL_NAME = "/home/ubuntu/victor-north-tx/models/Qwen2.5-0.5B-instruct"
# MODEL_NAME = "/home/ubuntu/victor-north-tx/models/Qwen2.5-3B-instruct"
# MODEL_NAME = "/home/ubuntu/victor-north-tx/models/Qwen2.5-7B-instruct"



def get_model_and_tokenizer(model_name, torch_dtype="auto", device_map="auto"):
    try:
        global model, tokenizer # Just to be sure that we don't have any old model or tokenizer
        del model, tokenizer
        print("Deleted existing model and tokenizer")
    except:
        pass
    start_time = time.time()
    # Load tokenizer and model (4-bit optional)
    print(f"Loading tokenizer and model for {model_name}")
    tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
    if tokenizer.chat_template is None:
        raise ValueError(f"Tokenizer for model {model_name} does not have a chat template.")
    else:
        print("Tokenizer has a chat template.")
    print(f"Tokenizer eos_token_id: {tokenizer.eos_token_id}, pad_token_id: {tokenizer.pad_token_id}")
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        torch_dtype=torch_dtype, # Automatically determines the best dtype (e.g., float16, bfloat16)
        device_map=device_map   # Automatically distributes the model across available devices (e.g., GPUs)
    )
    print(f"Model loaded in {time.time() - start_time:.2f} seconds")
    return model, tokenizer


model, tokenizer = get_model_and_tokenizer(MODEL_NAME, torch_dtype="auto", device_map="auto")

Deleted existing model and tokenizer
Loading tokenizer and model for /home/ubuntu/victor-north-tx/models/Qwen2.5-0.5B-instruct
Tokenizer has a chat template.
Tokenizer eos_token_id: 151645, pad_token_id: 151643
Model loaded in 1.03 seconds


In [ ]:
# Load gemnerate response function
# GPU_BACKEND = "cuda" if torch.cuda.is_available() else "cpu"
# GPU_BACKEND = "mps" if torch.cuda.is_available() else "cpu"

def generate_response_qwen(messages, tools, tokenizer, model, temperature=0.5, add_generation_prompt=True, enable_thinking=False, max_new_tokens=32768, verbose=False, gpu_backend=None):
    """
    Generate completion response for a given prompt.
    Args:
        messages: List of messages in the chat format.
        tools: List of tools to use for the completion.
        tokenizer: Tokenizer to use for the completion.
        model: Model to use for the completion.
        max_new_tokens: Maximum number of new tokens to generate.
    """
    prompt = tokenizer.apply_chat_template(messages, tools=tools, tokenize=False, add_generation_prompt=add_generation_prompt, enable_thinking=enable_thinking)
    if verbose:
        print(f"Prompt:\n{prompt}")
    if gpu_backend is not None:
        print("NOTE: If model was loaded using auto device map, you shouldn't specify gpu_backend")
    if gpu_backend is not None:
        model.to(gpu_backend)
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    # inputs.pop('token_type_ids', None)
    with torch.no_grad():
        outputs = model.generate(**inputs, 
                                  max_new_tokens=max_new_tokens,
                                  temperature=temperature
                                  )
        if verbose:
            print("*"*100)
            print(f"Full output:\n{tokenizer.decode(outputs[0], skip_special_tokens=False)}")
            print("*"*100)
    input_length = inputs.input_ids.shape[1]
    generated_token_ids = outputs[0][input_length:]
    if verbose:
        print("*"*100)
        print(f"Full generated output:\n{tokenizer.decode(generated_token_ids, skip_special_tokens=False)}")
        print("*"*100)
    # parsing thinking content
    try:
        # rindex finding 151668 (</think>)
        index = len(generated_token_ids) - generated_token_ids[::-1].index(151668)
    except ValueError:
        index = 0

    thinking_content = tokenizer.decode(generated_token_ids[:index], skip_special_tokens=True).strip("\n")
    content = tokenizer.decode(generated_token_ids[index:], skip_special_tokens=True).strip("\n")
    if verbose:
        print("*"*100)
        print(f"Thinking content: {thinking_content}")
        print(f"Content: {content}")
        print("*"*100)
    return content, thinking_content


test_messages = [
    {"role": "user", "content": "What is the capital of France?"}
]
test_tools = None
test_response, test_thinking_content = generate_response_qwen(test_messages, test_tools, tokenizer, model, add_generation_prompt=True, enable_thinking=False, verbose=True)
# print("thinking content:", test_thinking_content)
# print("content:", test_response)

# Parse tool calls
import json
import re

def parse_tool_calls_qwen(response: str) -> list[dict]:
    """
    Parse tool calls from a response.
    System instructions in Qwen2.5:
    ```
    For each function call, return a json object with function name and arguments within <tool_call></tool_call> XML tags:
    <tool_call>
    {"name": <function-name>, "arguments": <args-json-object>}
    </tool_call>
    ```
    """
    pattern = r"<tool_call>(.*?)</tool_call>"
    matches = re.findall(pattern, response, re.DOTALL)
    tool_calls = []
    
    for match in matches:
        # Process each line of the tool call content
        lines = match.strip().split('\n')
        for line in lines:
            line = line.strip()
            if line:  # Skip empty lines
                try:
                    tool_call = json.loads(line)
                    tool_calls.append(tool_call)
                except json.JSONDecodeError:
                    continue  # Skip lines that aren't valid JSON
    
    return tool_calls

print("Testing parse_tool_calls_qwen")
test_response = "I'm thinking about the capital of France. <tool_call>{\"name\": \"get_capital\", \"arguments\": {\"country\": \"France\"}}</tool_call>"
tool_calls = parse_tool_calls_qwen(test_response)
print(tool_calls)



    


Prompt:
<|im_start|>system
You are Qwen, created by Alibaba Cloud. You are a helpful assistant.<|im_end|>
<|im_start|>user
What is the capital of France?<|im_end|>
<|im_start|>assistant

****************************************************************************************************
Full output:
<|im_start|>system
You are Qwen, created by Alibaba Cloud. You are a helpful assistant.<|im_end|>
<|im_start|>user
What is the capital of France?<|im_end|>
<|im_start|>assistant
The capital of France is Paris.<|im_end|>
****************************************************************************************************
****************************************************************************************************
Full generated output:
The capital of France is Paris.<|im_end|>
****************************************************************************************************
****************************************************************************************************
Thinking c

In [10]:
def create_partial_trajectories(datapoint: OpenAICompletionDataPoint) -> list[OpenAICompletionDataPoint]:
    messages = datapoint.messages
    tools = datapoint.tools
    partial_trajectories = []
    for i, message in enumerate(messages):
        if message["role"] == "assistant":

            partial_trajectories.append(OpenAICompletionDataPoint(messages=messages[:i+1], tools=tools, parallel_tool_calls=datapoint.parallel_tool_calls))
    return partial_trajectories

partial_trajectories = create_partial_trajectories(dataset[0])

def replay(datapoint, max_new_tokens=32768, enable_thinking=False, verbose=False, add_generation_prompt=True):
    partial_trajectories = create_partial_trajectories(datapoint)
    for i, partial_trajectory in enumerate(partial_trajectories):
        original_response = partial_trajectory.messages[-1]
        convo_history = partial_trajectory.messages[:-1]
        start_time = time.perf_counter()

        replay_response, thinking_content = generate_response_qwen(convo_history, partial_trajectory.tools, tokenizer, model, add_generation_prompt=add_generation_prompt, max_new_tokens=max_new_tokens, enable_thinking=enable_thinking, verbose=verbose)
        print("=" * 80)
        print(f"🎯 PARTIAL TRAJECTORY {i} ({len(partial_trajectory.messages)} messages).\nGeneration time: {time.perf_counter() - start_time:.2f} seconds")
        print("=" * 80)
        
        print("\n📝 ORIGINAL RESPONSE:")
        print("-" * 40)
        print(original_response.get('content', "No content"))
        
        # Show original tool calls
        if original_response.get('tool_calls'):
            print("\n🛠️ ORIGINAL TOOL CALLS:")
            print("-" * 40)
            for j, tool_call in enumerate(original_response.get('tool_calls', [])):
                print(f"  {j+1}. {tool_call['function']['name']}: {tool_call['function']['arguments']}")
        else:
            print("\n🛠️ ORIGINAL TOOL CALLS: None")
        
        if thinking_content:
            print("\n💭 THINKING CONTENT:")
            print("-" * 40)
            print(thinking_content)
        
        print("\n🔄 REPLAY RESPONSE:")
        print("-" * 40)
        print(replay_response)
        
        tool_calls = parse_tool_calls_qwen(replay_response)
        if tool_calls:
            print("\n🛠️ REPLAY TOOL CALLS:")
            print("-" * 40)
            for j, tool_call in enumerate(tool_calls):
                print(f"  {j+1}. {tool_call['name']}: {tool_call['arguments']}")
        else:
            print("\n🛠️ REPLAY TOOL CALLS: None")
        
        print("\n")

replay(dataset[0])

🎯 PARTIAL TRAJECTORY 0 (2 messages).
Generation time: 1.09 seconds

📝 ORIGINAL RESPONSE:
----------------------------------------
Hi! How can I help you today?

🛠️ ORIGINAL TOOL CALLS: None

🔄 REPLAY RESPONSE:
----------------------------------------
```json
{
  "name": "book_reservation",
  "arguments": {
    "flight_number": "HAT001",
    "date": "2024-05-01"
  }
}
```

🛠️ REPLAY TOOL CALLS: None


🎯 PARTIAL TRAJECTORY 1 (4 messages).
Generation time: 0.85 seconds

📝 ORIGINAL RESPONSE:
----------------------------------------
To assist you with changing your flight, I need to know your user ID and the reservation ID for the flight you want to change. Could you please provide those? If you don't know your reservation ID, I can help you locate it.

🛠️ ORIGINAL TOOL CALLS: None

🔄 REPLAY RESPONSE:
----------------------------------------
Sure thing! Could you please tell me your preferred departure and arrival dates? Also, let me know if there are any special requirements like early mor

## Training

In [24]:
# Load dataset
from experiments.model_training.prepare_dataset import OpenAICompletionDataPoint
import json
dataset_path = "data/test_full-v1.jsonl"
dataset = []
with open(dataset_path, "r") as f:
    for line in f:
        datapoint = OpenAICompletionDataPoint.model_validate_json(line)
        dataset.append(datapoint)
print(f"Loaded {len(dataset)} datapoints")

Loaded 1107 datapoints


In [29]:
from datasets import Dataset

train_dataset = Dataset.from_list([dp.model_dump() for dp in dataset])

In [33]:
# from experiments.model_training.prepare_dataset import remove_none_values

print(json.dumps(train_dataset[0]["tools"][0], indent=4))

# remove_none_values(train_dataset[0])

# print(json.dumps(train_dataset[0]["tools"][0], indent=4))


{
    "function": {
        "description": "Book a reservation.",
        "name": "book_reservation",
        "parameters": {
            "$defs": {
                "APNNames": null,
                "APNSettings": null,
                "FlightInfo": {
                    "properties": {
                        "date": {
                            "description": "The date for the flight in the format 'YYYY-MM-DD', such as '2024-05-01'.",
                            "title": "Date",
                            "type": "string"
                        },
                        "flight_number": {
                            "description": "Flight number, such as 'HAT001'.",
                            "title": "Flight Number",
                            "type": "string"
                        }
                    },
                    "required": [
                        "flight_number",
                        "date"
                    ],
                    "title": "FlightInfo",

In [ ]:
from trl import SFTConfig, SFTTrainer
from transformers import EarlyStoppingCallback
from peft import LoraConfig, TaskType, get_peft_model

MODEL_NAME = "/home/ubuntu/victor-north-tx/models/Qwen2.5-0.5B-instruct"


## NOTE
## Tabular backends like Arrow/Parquet insert `None` for mismatched keys in nested structures.
## Those are cleaned up by SFTTraining using remove_non_values
## https://github.com/huggingface/trl/blob/30576d2ddcf2c0e17c399399e2465fbe81446ade/trl/trainer/sft_trainer.py#L71
## Pushed to prepare_dataset.py -> remove_none_values
## Warning: Those None values will make generation fail if not cleaned up.

output_dir = "./data/SFTcheckpoints"

USE_PEFT = True
if USE_PEFT:
    learning_rate = 1e-4 # Higher learning rate for PEFT?
else:
    learning_rate = 8e-5

sft_config = SFTConfig(
    assistant_only_loss=True,                # Only compute the loss on the assistant messages
    report_to="none",                        # disable logging to W&B
    logging_strategy="steps",
    learning_rate=learning_rate,                      # Learning rate for training. 
    num_train_epochs=20,                     #  Set the number of epochs to train the model.
    per_device_train_batch_size=2,           # Batch size for each device (e.g., GPU) during training. 
    gradient_accumulation_steps=8,           # Number of steps before performing a backward/update pass to accumulate gradients.
    gradient_checkpointing=True,             # Enable gradient checkpointing to reduce memory usage during training at the cost of slower training speed.
    logging_steps=2,                         # Frequency of logging training progress (log every 2 steps).
    eval_strategy="epoch",                   # evaluate at end of each epoch
    save_strategy="epoch",                   # save checkpoint at end of each epoch
    save_total_limit=1,                      # keep only the best/latest model
    load_best_model_at_end=True,             # load best model according to eval loss
    metric_for_best_model="eval_loss",       # use eval loss for best model selection
    greater_is_better=False,                 # lower eval_loss is better
    output_dir=output_dir               # directory to save checkpoints
)


# Instantiate early stopping callback
early_stopping_callback = EarlyStoppingCallback(
    early_stopping_patience=2  # Stop if no improvement for 2 evals (epochs)
)

tokenizer, model = get_model_and_tokenizer(MODEL_NAME, torch_dtype="auto", device_map="auto")

if USE_PEFT: # FIXME: Check what's the right config.
    # lora_config = LoraConfig(
    #     r=64,
    #     lora_alpha=16,
    #     target_modules=["c_attn", "q_proj", "v_proj"],  # adjust to Qwen architecture
    #     lora_dropout=0.05,
    #     bias="none",
    #     task_type=TaskType.CAUSAL_LM,
    # )
    lora_config = LoraConfig()
else:
    lora_config = None

sft_trainer = SFTTrainer(
    model=model,
    args=sft_config,
    train_dataset=train_dataset,
    processing_class=tokenizer,
    callbacks=[early_stopping_callback],
    peft_config=lora_config 
    
)


ValueError: Your setup doesn't support bf16/gpu.